https://stackoverflow.com/questions/714063/importing-modules-from-parent-folder

In [1]:
import sys
sys.path.insert(0, '..')

In [2]:
import alpha_vantage

In [3]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import time
from operator import methodcaller
import requests
import datetime
import yaml
import os
import csv
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import traceback
import sys

In [4]:
tsdaily_dfs, pe_ratio_dfs = {}, {}

In [5]:
symbols = ['AAPL','AXP','GOOG','META','NVDA','AMZN','PM','SBUX','DIS','MSFT','NFLX','ICE','PLTR']
symbols = symbols + ['SPOT','NKE','WMT','JNJ','SHOP','TSLA','MA','XOM','V','HD','AMD','PEP','MCD','ABT','IBM','UPS','MO','STZ']

In [6]:
tsdaily_dfs.keys()

dict_keys([])

In [7]:
for symbol in symbols:
  if symbol not in tsdaily_dfs.keys() or symbol not in pe_ratio_dfs.keys():
    try:
      tsdaily_dfs[symbol], pe_ratio_dfs[symbol] = alpha_vantage.generate_dfs(symbol)
    except KeyError:
      print(f'Failed at {alpha_vantage.get_prev_api_key()}')
      print(traceback.format_exc())
      break

PLTR
11


In [8]:
# To CSVs
# for symbol in symbols:
for symbol in tsdaily_dfs.keys():
    tsdaily_dfs[symbol].to_csv(f'../tsdaily_dfs/{symbol}.csv')
for symbol in pe_ratio_dfs.keys():
    pe_ratio_dfs[symbol].to_csv(f'../pe_ratio_dfs/{symbol}.csv')

In [7]:
# Read CSVs
# for symbol in symbols:
for symbol in [i[:-4] for i in os.listdir('../tsdaily_dfs')]:
    tsdaily_df = pd.read_csv(f'../tsdaily_dfs/{symbol}.csv',index_col=0)
    tsdaily_df.index = pd.to_datetime(tsdaily_df.index)
    tsdaily_dfs[symbol] = tsdaily_df

for symbol in [i[:-4] for i in os.listdir('../pe_ratio_dfs')]:
    pe_ratio_df = pd.read_csv(f'../pe_ratio_dfs/{symbol}.csv',index_col=0)
    pe_ratio_df.index = pd.to_datetime(pe_ratio_df.index)
    for col in pe_ratio_df.columns[:1]:
        pe_ratio_df[col] = pd.to_datetime(pe_ratio_df[col], errors='coerce')
    pe_ratio_dfs[symbol] = pe_ratio_df

In [8]:
bollinger_returns= {}
net_change = {}
last_price = {}
for symbol in tsdaily_dfs.keys():
  bollinger_returns[symbol] = alpha_vantage.generate_bollinger_returns(alpha_vantage.create_buy_sell_pairs(tsdaily_dfs, symbol, start_date = '2019-01-01'))
  net_change[symbol] = tsdaily_dfs[symbol].iloc[-1,3] - tsdaily_dfs[symbol].iloc[0,3]
  last_price[symbol] = tsdaily_dfs[symbol].iloc[-1,3]

In [9]:
metrics = pd.DataFrame([bollinger_returns,net_change,last_price],).transpose().rename(columns={0:'bollinger_returns',1:'net_change',2:'last_price'})
# metrics['net_returns'] = metrics['bollinger_returns']/metrics['net_change']*100
metrics['net_over_change'] = metrics['bollinger_returns']/metrics['net_change'].abs()*100
metrics['net_over_price'] = metrics['bollinger_returns']/metrics['last_price']*100
metrics = metrics.sort_values(by='net_over_price',ascending=False)
metrics

,bollinger_returns,net_change,last_price,net_over_change,net_over_price
AMZN,1122.00,84.2100,153.34,1332.383327,731.707317
GOOG,795.15,-417.0200,141.44,190.674308,562.181844
TSLA,293.59,214.5600,238.45,136.833520,123.124345
MA,255.94,372.7700,418.77,68.658959,61.117081
ABT,60.62,69.2100,109.52,87.588499,55.350621
AAPL,98.47,115.5300,193.15,85.233273,50.981103
V,128.89,201.4800,257.98,63.971610,49.961237
PEP,71.81,139.3900,172.95,51.517325,41.520671
MSFT,148.52,281.7000,374.07,52.722755,39.703799
WMT,62.38,101.4000,157.65,61.518738,39.568665


In [ ]:
summary_info = {}
for name in tsdaily_dfs.keys():
    # print(name)
    info = alpha_vantage.pull_basic_data(name)
    if len(info.keys()) <= 2:
        print(f'stopped at {name}')
        break
    summary_info[name] = alpha_vantage.pull_basic_data(name)

In [19]:
summary_info = {k:v for k,v in summary_info.items() if len(v.keys()) > 2}
summary_info.keys()

dict_keys(['V', 'MO', 'STZ', 'PEP', 'SBUX', 'MCD', 'AMZN', 'UPS', 'MSFT', 'AMD', 'NVDA', 'HD', 'AXP', 'XOM', 'ICE', 'NKE', 'IBM', 'PM', 'DIS', 'NFLX', 'ABT', 'TSLA', 'META', 'SHOP'])

In [24]:
# To CSV
overview_df = pd.DataFrame(summary_info).transpose()
overview_df.to_csv('../summary_info/company_overview.csv')

In [27]:
# Read CSV
overview_df = pd.read_csv('../summary_info/company_overview.csv', index_col=0)

In [16]:
metrics.index

Index(['GOOG', 'MA', 'XOM', 'PLTR', 'MSFT', 'V', 'MCD', 'PEP', 'AXP', 'AAPL',
       'IBM', 'UPS', 'ABT', 'SBUX', 'WMT', 'STZ', 'ICE', 'JNJ', 'MO', 'PM',
       'HD', 'SPOT', 'AMD', 'DIS', 'NFLX', 'META', 'NVDA', 'NKE', 'TSLA',
       'AMZN', 'SHOP'],
      dtype='object')

In [10]:
bollinger_returns= {}
net_change = {}
last_price = {}
for symbol in tsdaily_dfs.keys():
  bollinger_returns[symbol] = alpha_vantage.generate_bollinger_returns(alpha_vantage.create_buy_sell_pairs(tsdaily_dfs, symbol, start_date = '2021-01-01'))
  net_change[symbol] = tsdaily_dfs[symbol].iloc[-1,3] - tsdaily_dfs[symbol].iloc[0,3]
  last_price[symbol] = tsdaily_dfs[symbol].iloc[-1,3]
metrics = pd.DataFrame([bollinger_returns,net_change,last_price],).transpose().rename(columns={0:'bollinger_returns',1:'net_change',2:'last_price'})
# metrics['net_returns'] = metrics['bollinger_returns']/metrics['net_change']*100
metrics['net_over_change'] = metrics['bollinger_returns']/metrics['net_change'].abs()*100
metrics['net_over_price'] = metrics['bollinger_returns']/metrics['last_price']*100
metrics = metrics.sort_values(by='net_over_price',ascending=False)
metrics

,bollinger_returns,net_change,last_price,net_over_change,net_over_price
GOOG,477.40,-417.0200,141.44,114.478922,337.528281
MA,179.02,372.7700,418.77,48.024251,42.749003
XOM,30.60,28.4700,103.22,107.481560,29.645418
PLTR,4.25,6.7500,16.25,62.962963,26.153846
MSFT,95.89,281.7000,374.07,34.039759,25.634240
V,62.78,201.4800,257.98,31.159420,24.335220
MCD,71.06,252.7000,294.39,28.120301,24.138048
PEP,35.13,139.3900,172.95,25.202669,20.312229
AXP,36.17,38.0600,187.06,95.034157,19.336042
AAPL,33.18,115.5300,193.15,28.719813,17.178359


In [32]:
alpha_vantage.plot_graph(tsdaily_dfs, pe_ratio_dfs, 'GOOG', start_date = '2021-01-01')

In [31]:
alpha_vantage.plot_graph(tsdaily_dfs, pe_ratio_dfs, 'GOOG', start_date = '2022-8-01')

In [29]:
alpha_vantage.plot_graph(tsdaily_dfs, pe_ratio_dfs, 'AMZN', start_date = '2021-01-01')

In [14]:
alpha_vantage.plot_graph(tsdaily_dfs, pe_ratio_dfs, 'MCD', start_date = '2019-01-01')

In [15]:
alpha_vantage.plot_graph(tsdaily_dfs, pe_ratio_dfs, 'PLTR', start_date = '2019-01-01')